In [1]:
# Before we begin, we supress deprecation warnings resulting from nltk on Kaggle
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google import colab
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/FYP

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/FYP


# Linear SVM classification of sentiment in tweets about airlines

This notebook describes an attempt to classify tweets by sentiment. It describes the initial data exploration, as well as implementation of a linear one-vs-rest Support-Vector-Machine (SVM) classifier.

## What is in the dataset?

It's always good to start by exploring the data that we have available. To do this we load the raw csv file using [Pandas][1] and check what the columns are.

  [1]: http://pandas.pydata.org/

In [4]:
import pandas as pd
tweets = pd.read_csv("cyberbullying_tweets.csv")
list(tweets.columns.values)

['TWEET_TEXT',
 'CYBERBULLYING_TYPE',
 'AGE',
 'GENDER',
 'RELIGION',
 'ETHNICITY',
 'OTHER',
 'RESULT']

We want to be able to determine the sentiment of a tweet without any other information but the tweet text itself, hence the 'text' column is our focus. Using the text we are going to try and predict 'airline_sentiment'.

First we take a look at what a typical record looks like.

In [5]:
tweets.head()

,TWEET_TEXT,CYBERBULLYING_TYPE,AGE,GENDER,RELIGION,ETHNICITY,OTHER,RESULT
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0,0,0,0,0,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0,0,0,0,0,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0,0,0,0,0,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0,0,0,0,0,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0,0,0,0,0,0


Now lets take a look at what sentiments have been found.

It turns out that our dataset is unbalanced with significantly more negative than positive tweets. We will focus on the issue of identifying negative tweets, and hence treat neutral and positive as one class. It's good to keep in mind that, while a terrible classifier, if we always guessed a tweet was negative we'd be right 62.7% of the time (9178 of 14640). That clearly wouldn't be a very useful classifier, but worth to remember.

# What characterizes text of different sentiments?

While we still haven't decided what classification method to use, it's useful to get an idea of how the different texts look. This might be an "old school" approach in the age of deep learning, but lets indulge ourselves nevertheless. 

To explore the data we apply some crude preprocessing. We will tokenize and lemmatize using [Python NLTK][1], and transform to lower case. As words mostly matter in context we'll look at bi-grams instead of just individual tokens.

As a way to simplify later inspection of results we will store all processing of data together with it's original form. This means we will extend the Pandas dataframe into which we imported the raw data with new columns as we go along.

### Preprocessing
Note that we remove the first two tokens as they always contain "@ airline_name". We begin by defining our normalization function.


  [1]: http://www.nltk.org/

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import re, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

def normalizer(tweet):
    only_letters = re.sub("[^a-zA-Z]", " ",tweet) 
    tokens = nltk.word_tokenize(only_letters)[2:]
    lower_case = [l.lower() for l in tokens]
    filtered_result = list(filter(lambda l: l not in stop_words, lower_case))
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
    return lemmas

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
normalizer("Here is text about an airline I like.")

['text', 'airline', 'like']

In [17]:
pd.set_option('display.max_colwidth', -1) # Setting this so we can see the full content of cells
tweets['normalized_tweet'] = tweets.TWEET_TEXT.apply(normalizer)
tweets[['TWEET_TEXT','normalized_tweet']].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,TWEET_TEXT,normalized_tweet
0,"In other words #katandandre, your food was crapilicious! #mkr","[word, katandandre, food, crapilicious, mkr]"
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio, neighbour, wonderlandten, etc]"
2,@XochitlSuckkks a classy whore? Or more red velvet cupcakes?,"[classy, whore, red, velvet, cupcake]"
3,"@Jason_Gio meh. :P thanks for the heads up, but not too concerned about another angry dude on twitter.","[meh, p, thanks, head, concerned, another, angry, dude, twitter]"
4,"@RudhoeEnglish This is an ISIS account pretending to be a Kurdish account. Like Islam, it is all lies.","[isi, account, pretending, kurdish, account, like, islam, lie]"


In [18]:
from nltk import ngrams
def ngrams(input_list):
    #onegrams = input_list
    bigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:]))]
    trigrams = [' '.join(t) for t in list(zip(input_list, input_list[1:], input_list[2:]))]
    return bigrams+trigrams
tweets['grams'] = tweets.normalized_tweet.apply(ngrams)
tweets[['grams']].head()

,grams
0,"[word katandandre, katandandre food, food crapilicious, crapilicious mkr, word katandandre food, katandandre food crapilicious, food crapilicious mkr]"
1,"[aussietv white, white mkr, mkr theblock, theblock imacelebrityau, imacelebrityau today, today sunrise, sunrise studio, studio neighbour, neighbour wonderlandten, wonderlandten etc, aussietv white mkr, white mkr theblock, mkr theblock imacelebrityau, theblock imacelebrityau today, imacelebrityau today sunrise, today sunrise studio, sunrise studio neighbour, studio neighbour wonderlandten, neighbour wonderlandten etc]"
2,"[classy whore, whore red, red velvet, velvet cupcake, classy whore red, whore red velvet, red velvet cupcake]"
3,"[meh p, p thanks, thanks head, head concerned, concerned another, another angry, angry dude, dude twitter, meh p thanks, p thanks head, thanks head concerned, head concerned another, concerned another angry, another angry dude, angry dude twitter]"
4,"[isi account, account pretending, pretending kurdish, kurdish account, account like, like islam, islam lie, isi account pretending, account pretending kurdish, pretending kurdish account, kurdish account like, account like islam, like islam lie]"


And now some counting.

In [19]:
import collections
def count_words(input):
    cnt = collections.Counter()
    for row in input:
        for word in row:
            cnt[word] += 1
    return cnt

In [22]:
tweets[(tweets.RESULT == 1)][['grams']].apply(count_words)['grams'].most_common(20)

[('high school', 4561),
 ('rape joke', 2151),
 ('http co', 1911),
 ('dumb as', 1505),
 ('as nigger', 1326),
 ('bullied high', 1324),
 ('bullied high school', 1317),
 ('dumb as nigger', 1193),
 ('school bully', 1165),
 ('dumb nigger', 1136),
 ('girl bullied', 1093),
 ('dumb fuck', 1014),
 ('obama dumb', 981),
 ('obama dumb as', 965),
 ('islamic terrorism', 863),
 ('christian woman', 809),
 ('gay joke', 770),
 ('fuck obama', 750),
 ('fuck obama dumb', 739),
 ('tayyoung fuck', 638)]

We can already tell there's a pattern here. Sentences like "cancelled flight", "late flight", "booking problems",  "delayed flight" stand out clearly. Lets check the positive tweets.

In [23]:
tweets[(tweets.RESULT == 0)][['grams']].apply(count_words)['grams'].most_common(20)

[('http co', 960),
 ('kat andre', 95),
 ('mkr mkr', 75),
 ('gt gt', 69),
 ('mkr http', 57),
 ('mkr http co', 56),
 ('bullying http', 47),
 ('gon na', 46),
 ('instant restaurant', 46),
 ('gt gt gt', 46),
 ('amp andre', 41),
 ('kat amp', 39),
 ('kat amp andre', 37),
 ('bullying se', 35),
 ('bullying http co', 31),
 ('de bullying', 30),
 ('bit ly', 29),
 ('high school', 29),
 ('http bit', 28),
 ('http bit ly', 28)]

Some more good looking patterns here. We can however see that with 3-grams clear patterns are rare. "great customer service" occurs 12 times in 2362 positive responses, which really doesn't say much in general. 

Satisfied that our data looks possible to work with begin to construct our first classifier.

# Linear SVM classifier
We will build a simple, linear Support-Vector-Machine (SVM) classifier. The classifier will take into account each unique word present in the sentence, as well as all consecutive words. To make this representation useful for our SVM classifier we transform each sentence into a vector. The vector is of the same length as our vocabulary, i.e. the list of all words observed in our training data, with each word representing an entry in the vector. If a particular word is present, that entry in the vector is 1, otherwise 0.

To create these vectors we use the CountVectorizer from [sklearn][1]. 


  [1]: http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

## Preparing the data

In [24]:
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [26]:
vectorized_data = count_vectorizer.fit_transform(tweets.TWEET_TEXT)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))

In [34]:
targets = tweets.RESULT

To check performance of our classifier we want to split our data in to train and test.

In [35]:
from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

## Fitting the classifier

We're now ready to fit the classifier to our data. We'll spend more time on hyper parameter tuning later, so for now we just pick some reasonable guesses. Note here that we use the OneVsRestClassifier. This allows us to get the probability distribution over all three classes. Behind the scenes we actually create three classifiers. Each of these classifiers determines the probability that the datapoint belongs to it's corresponding class, or any of the other classes. Hence the name OneVsRest.

In [36]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf_output = clf.fit(data_train, targets_train)

## Evaluation of results

In [37]:
clf.score(data_test, targets_test)

0.837657912669707

It's most likely possible to achieve a higher score with more tuning, or a more advanced approach. Lets check on how it does on a couple of sentences.

In [ ]:
sentences = count_vectorizer.transform([
    "What a great airline, the trip was a pleasure!",
    "My issue was quickly resolved after calling customer support. Thanks!",
    "What the hell! My flight was cancelled again. This sucks!",
    "Service was awful. I'll never fly with you again.",
    "You fuckers lost my luggage. Never again!",
    "I have mixed feelings about airlines. I don't know what I think.",
    ""
])
clf.predict_proba(sentences)

So while results aren't very impressive overall, we can see that it's doing a good job on these obvious sentences. 

## What is hard for the classifier?

It's interesting to know which sentences are hard. To find out, lets apply the classifier to all our test sentences and sort by the marginal probability.

Here are some of the hardest sentences.

In [ ]:
predictions_on_test_data = clf.predict_proba(data_test)
index = np.transpose(np.array([range(0,len(predictions_on_test_data))]))
indexed_predictions = np.concatenate((predictions_on_test_data, index), axis=1).tolist()

In [ ]:
def marginal(p):
    top2 = p.argsort()[::-1]
    return abs(p[top2[0]]-p[top2[1]])
margin = sorted(list(map(lambda p : [marginal(np.array(p[0:3])),p[3]], indexed_predictions)), key=lambda p : p[0])
list(map(lambda p : tweets.iloc[data_test_index[int(p[1])].toarray()[0][0]].text, margin[0:10]))

and their probability distributions?

In [ ]:
list(map(lambda p : predictions_on_test_data[int(p[1])], margin[0:10]))

How about the easiest sentences?

In [ ]:
list(map(lambda p : tweets.iloc[data_test_index[int(p[1])].toarray()[0][0]].text, margin[-10:]))

and their probability distributions?

In [ ]:
list(map(lambda p : predictions_on_test_data[int(p[1])], margin[-10:]))

Looks like all of the easiest sentences are negative. What is the distribution of certainty across all sentences?

In [ ]:
import matplotlib.pyplot as plt
marginal_probs = list(map(lambda p : p[0], margin))
n, bins, patches = plt.hist(marginal_probs, 25, facecolor='blue', alpha=0.75)
plt.title('Marginal confidence histogram - All data')
plt.ylabel('Count')
plt.xlabel('Marginal confidence')
plt.show()

Lets break it down by positive and negative sentiment to see if one is harder than the other.

### Positive data

In [ ]:
positive_test_data = list(filter(lambda row : row[0]==2, hstack((targets_test[:,None], data_test)).toarray()))
positive_probs = clf.predict_proba(list(map(lambda r : r[1:], positive_test_data)))
marginal_positive_probs = list(map(lambda p : marginal(p), positive_probs))
n, bins, patches = plt.hist(marginal_positive_probs, 25, facecolor='green', alpha=0.75)
plt.title('Marginal confidence histogram - Positive data')
plt.ylabel('Count')
plt.xlabel('Marginal confidence')
plt.show()

### Neutral data

In [ ]:
positive_test_data = list(filter(lambda row : row[0]==1, hstack((targets_test[:,None], data_test)).toarray()))
positive_probs = clf.predict_proba(list(map(lambda r : r[1:], positive_test_data)))
marginal_positive_probs = list(map(lambda p : marginal(p), positive_probs))
n, bins, patches = plt.hist(marginal_positive_probs, 25, facecolor='blue', alpha=0.75)
plt.title('Marginal confidence histogram - Neutral data')
plt.ylabel('Count')
plt.xlabel('Marginal confidence')
plt.show()

### Negative data

In [ ]:
negative_test_data = list(filter(lambda row : row[0]==0, hstack((targets_test[:,None], data_test)).toarray()))
negative_probs = clf.predict_proba(list(map(lambda r : r[1:], negative_test_data)))
marginal_negative_probs = list(map(lambda p : marginal(p), negative_probs))
n, bins, patches = plt.hist(marginal_negative_probs, 25, facecolor='red', alpha=0.75)
plt.title('Marginal confidence histogram - Negative data')
plt.ylabel('Count')
plt.xlabel('Marginal confidence')
plt.show()

Clearly the positive data is much harder for the classifier. This makes sense since there's a lot less of it. An important challenge in building a better classifier will then be how to handle positive data.

A more advanced classifier is described here.